In [ ]:
!pip install sidetable

< updated the waterfall and sankey charts >
    
# Introduction

Inspiration for this notebook came from three places. One is to master the visualisation libraries, next is to learn story telling with the data, and finally the strong urge to match the visualisation that has been created using other BI/ Visualisation tool. 

# Layout

The notebook will have simple pattern. The powerBI/ Excel tool charts carry so much space, so I have removed them. The notebook cells are independent, so each cell can render the visual by themselves, after couple of initial cells are executed. The visual heading provides the info on the chart type. 

# Purpose

To achieve mastery, lot of training is required. The silver-bullet is getting feedback. This Data visualisation exercise will provide the necessary training. The feedback can be seen in the way graphs and charts render, and share the story that is hidden inside the data.

Lets dive in to the Kaggle survey 2021 and 2020 data keyboard first.

# What you will Learn???

1) Building Gorgeous Sankey Charts

2) Waterfall charts using dataframes, and plotly

3) Slicing and Dicing the datasets like a Pro (Inspired by PowerBI & Tableau Chart notebook)

4) Relate to Funnel chart with ease

5) Bring Multiple charts on to a single frame using Graph_objects 

last but not the least, 

6) Learning to creat links in the within notebook to send the readers to "exotic" destinations in notebook (It took me 3 versions saving to understand it correctly and implement)

7) Writing long code sections by stitching & commenting the experiments done on code snippets in seperate cells(You will see this, for many complicated looking charts. I did this to share my strategy to those new to Visualisation with Python and plotly)

Rare charts like Funnel, Sankey and Waterfall charts are seldom used by many new users. Stories these charts tell can be spell bindng. (click on [Visual 21](#vis_21) for a sneek peek). I felt, it is due to the lack of examples,that triggers ones imagination. I got the imagination from @Rahman notebook here.
https://www.kaggle.com/kalilurrahman/2021-kaggle-mlds-survey-a-perspective?kernelSessionId=79811250

The charts rendered using PowerBI & Tableau is elegant, and must be easier to use. The challenge is when you really want to add some tweak, or a story anchor inside the chart. Plotly and Python's Pandas can be excellent work horses. Even in a hands of layman, with basic programming knowledge. 

This notebook can be a quick reference. The code is not "function" friendly. It took some time to write the graphing commands directly. When certain option needs to be added, creates redundant complexity. You will find lot of tid-bits strewn in the code written for the plots... Enjoy

# <a id="cont">Contents</a>

PS: Introduction, Purpose and the learnings are hidden in above cell, unhide to see the same...

1. [Visual-1 : Distribution of the Kagglers by Gender and Age](#vis_1)
2. [Visual-2 : Participating Nations](#vis_2)
3. [Visual 3: Global Participation Based on Gender : Choropleth Map](#vis_3)
4. [Visual-4 : Distribution of Software & Education](#vis_4)
5. [Visual_5: Distribution of Profession with respect to Age](#vis_5)
6. [Visual_6: Distribution of Machine Learning Experience](#vis_6)
7. [Visual 7: Salary Range](#vis_7)
8. [Visual_8 Salary Distribution based on the profession](#vis_8)
9. [Visual 9: Kagglers and their Industry types](#vis_9)
10. [Visual 10: Primary Databased tools use Distribution](#vis_10)
11. [Visual 11 : Distribution of Business Intelligence tools](#vis_11)
12. [Visual 12 : Population Distribution with the rendering of Table in same chart](#vis_12)
13. [Visual 13: Global map of Kaggle participants](#vis_13)
14. [Visual 14 : Treemap of the Kaggler distribution based on the Gender](#vis_14)
15. [Visual 15: Funnel Chart of Country citizen participation](#vis_15)
16. [Visual 16 : Mix of Treemap, Ribbon chart, Funnel and Pie charts](#vis_16)
17. [Visual 17 : Subplots of multiple demographic spread](#vis_17)
18. [Visual 18: Count of Employment and Gender by Country and Experience](#vis_18)
19. [Visual 19: Trying the waterfall Chart (Will add Dropdown option for the countries)](#vis_19)
20. [Visual 20: Sankey Chart to see the flow of Kagglers in Gender](#vis_20)
21. [Visual 21: Continuing with Sankey Chart with multiple Levels](#vis_21)
22. [Visual 22: Cloud computing usage Distribution](#vis_22)

The data creation step for the chart is laborious, if not planned from the beginning. Once we plan the sankey chart is a breath taking visual

In [ ]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
import plotly.graph_objects as go
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 5000)
import warnings
warnings.filterwarnings("ignore")
#os.mkdir('/kaggle/working/individual_charts/')
import matplotlib.pyplot as plt
# Load the data
#Will come in handy to wrap the lengthy texts
import textwrap
#useful libraries and functions
import sidetable as stb
from itertools import repeat
#Libraries that give a different visual possibilities
from pandas import option_context 
from plotly.subplots import make_subplots

def long_sentences_seperate(sentence, width=30):
    try:
        splittext = textwrap.wrap(sentence,width)
        text = '<br>'.join(splittext)#whitespace is removed, and the sentence is joined
        return text
    except:
        return sentence

def load_csv(base_dir,file_name):
    """Loads a CSV file into a Pandas DataFrame"""
    file_path = os.path.join(base_dir,file_name)
    df = pd.read_csv(file_path,low_memory=False,encoding='ISO-8859-1')
    return df    

base_dir_2020 = '/kaggle/input/kaggle-survey-2020'
file_name_2020 = 'kaggle_survey_2020_responses.csv'
survey_df_2020 = load_csv(base_dir_2020,file_name_2020)
responses_df_2020 = survey_df_2020[1:]
survey_df_2020.to_csv('2020_kaggle_ds_and_ml_survey_responses_only.csv',index=False)

base_dir_2021 = '/kaggle/input/kaggle-survey-2021/'
file_name_2021 = 'kaggle_survey_2021_responses.csv'
survey_df_2021 = load_csv(base_dir_2021,file_name_2021)
responses_df_2021 = survey_df_2021[1:]
survey_df_2021.to_csv('2021_kaggle_ds_and_ml_survey_responses_only.csv',index=False)

print('Total Number of Responses 2020: ',responses_df_2020.shape[0])
print('Total Number of Responses 2021: ',responses_df_2021.shape[0])

In [ ]:
#Kagglers demographics

r_dgc_2021 = responses_df_2021.loc[:,['Q1','Q2','Q3','Q4','Q5','Q6','Q20','Q21','Q22',
                                      'Q25','Time from Start to Finish (seconds)']]
r_dgc_2020 = responses_df_2020.loc[:,['Q1','Q2','Q3','Q4','Q5','Q6','Q20','Q21','Q22',
                                      'Q25','Time from Start to Finish (seconds)']]
r_dgc_2021['Finish_time'] = r_dgc_2021['Time from Start to Finish (seconds)'].astype('int')
r_dgc_2020['Finish_time'] = r_dgc_2020['Time from Start to Finish (seconds)'].astype('int')

In [ ]:
#Idea is to groupby and count the number of occurences of the people with set of characters. This way the 
#demographics can be rendered quantitatively
dg_c_2021 = r_dgc_2021.groupby(['Q1','Q2','Q3','Q4','Q5','Q6','Q20','Q21','Q22','Q25'])['Finish_time'].count().reset_index()
dg_c_2020 = r_dgc_2020.groupby(['Q1','Q2','Q3','Q4','Q5','Q6','Q20','Q21','Q22','Q25'])['Finish_time'].count().reset_index()
dg_c_2021['year'] = 2021
dg_c_2020['year'] = 2020

In [ ]:
dg_c_2021.columns = ['Age','Gender','Country','Education','Employment','Experience','Industry','Total_Employee',
                     'Datascientists','salary','Numbers','Year']
dg_c_2020.columns = ['Age','Gender','Country','Education','Employment','Experience','Industry','Total_Employee',
                     'Datascientists','salary','Numbers','Year']

 [Back to Contents](#cont)

# <a id="vis_1"> Visual-1 : Distribution of the Kagglers by Gender and Age</a>

In [ ]:
#Need to get the counts based on the Gender, and age. 
visual_1_grp = dg_c_2021.groupby(['Age','Gender'])['Numbers'].sum().reset_index()
visual_1 = px.bar(data_frame=visual_1_grp,x='Gender',y='Numbers',color='Age',
                  barmode='group',title='Kagglers Age and Gender distribution')
visual_1.show()
del visual_1_grp #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_2"> Visual-2 : Participating Nations</a>

In [ ]:
visual_2_grp = dg_c_2020.groupby('Country')['Numbers'].sum().reset_index()
visual_2_grp.Country = visual_2_grp.Country.apply(lambda x : long_sentences_seperate(x,7))
visual_2 = px.treemap(data_frame=visual_2_grp,path=['Country'],names='Country',
                      values = 'Numbers',title='Participating Nations')
visual_2.show()
del visual_2_grp

 [Back to Contents](#cont)

# <a id="vis_3"> Visual 3: Global Participation Based on Gender : Choropleth Map</a> 

In [ ]:
visual_3_grp = dg_c_2021.groupby(['Country','Gender'])['Numbers'].sum().reset_index()
visual_3 = px.choropleth(visual_3_grp, locations="Country",color='Numbers',
                         locationmode='country names',animation_frame='Gender',
                        title = 'Participation based on the Countries & Gender')
visual_3.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
visual_3.show()
del visual_3_grp

 [Back to Contents](#cont)

# <a id="vis_4"> Visual-4 : Distribution of Software & Education</a> 

The above visual is showing the counts based on education level, and the following products usage. 
Will be needing an helper function to get the necessary replies collected, and then aggregate them. 

cols_counter() function is written for the same. The function takes the source_dataframe, name of the column that will be the base,the question number that we are looking to query and what category it will belong to.

Statistical programes used : can be found using the Q7 (Statistical Softwares)

IDEs used : Q9 (IDEs)

Cloud Computing platform : Q10 (Cloud Platforms)

Cloud Computing products: Q27 (Cloud Products)

Business Insight : Q34 (BI softwares)


In [ ]:
#Function takes the dataframe and groups and counts the number of values in each column, based on the "column"
#that is required for analysis. The counting column will be the 'Time from Start to Finish (seconds)' since 
#this cell will have some value.
def cols_counter(df,base_col,question_col,category):
    col_part = [] #Initialising the cols of interest
    for col in df.columns:
        if question_col in col:
            col_part.append(col) #till here same as group cols
    
    col_part.append('Time from Start to Finish (seconds)')
    col_part.append(base_col)
    temp_df = df[col_part]# creating temp dataframe to get only the replies that are required for the visual in question
    
    ret_df = pd.DataFrame()# initialising the dataframe that will contain the replies counted.
    
    for col_cnt in temp_df.columns: #Starting the loop for the counting and dataframe collection
        if ((col_cnt != base_col) and (col_cnt != 'Time from Start to Finish (seconds)')):
            temp_replies = temp_df.groupby([col_cnt,base_col])['Time from Start to Finish (seconds)'].count().reset_index()
            temp_replies.columns = ['Reply_Choices','base_col','Counts']
            temp_replies['category'] = category
        #col_vals.append(temp_edu[cols].unique()[0])
        ret_df = pd.concat([temp_replies,ret_df]) #appending to the returning df

    return ret_df

In [ ]:
from pandas import option_context #Helpful in seeing the questions that can help to answer

with option_context('display.max_colwidth', 400,'display.max_rows',600):
    display(survey_df_2021.iloc[:2,175:250].T)

In [ ]:
Q7_df = cols_counter(responses_df_2021,base_col='Q4',question_col='Q7',category='Statistical softwares')
Q9_df = cols_counter(responses_df_2021,base_col='Q4',question_col='Q9',category='IDEs')
Q10_df = cols_counter(responses_df_2021,base_col='Q4',question_col='Q10',category='Cloud Platforms')
Q27_df = cols_counter(responses_df_2021,base_col='Q4',question_col='Q27',category='Cloud Products')
Q34_df = cols_counter(responses_df_2021,base_col='Q4',question_col='Q34',category='BI softwares')

In [ ]:
Q32_A_df = cols_counter(responses_df_2021,base_col='Q4',question_col='Q32_A',category='Databases')

In [ ]:
visual_4_grp = pd.concat([Q34_df,Q27_df,Q10_df,Q9_df,Q7_df]) # Concatenate the choices
visual_4_grp = visual_4_grp[~(visual_4_grp.Reply_Choices == 'None')] #remove the None value choices,since they don't add

In [ ]:
visual_4 = px.bar(data_frame=visual_4_grp,x='Counts',color='base_col',y='category',
                 title = 'Distribution of Software & Education',height=500)

visual_4.update_layout(legend=dict(x=0,y=-1.5,traceorder="reversed",
                                   title_font_family="Times New Roman",
                                   font=dict(family="Courier",size=12,color="black"),
                                   bgcolor="white",
                                   bordercolor="Black",
                                   borderwidth=2),
                      yaxis={'categoryorder':'total descending'})
visual_4.show()
del visual_4_grp

 [Back to Contents](#cont)

# <a id="vis_5">Visual_5: Distribution of Profession with respect to Age</a>

In [ ]:
visual_grp_5 = dg_c_2021.groupby(['Age','Employment'])['Numbers'].sum().reset_index()
visual_5 = px.bar(data_frame=visual_grp_5,y='Employment',x='Numbers',color='Age',
                  barmode='stack',title='Kagglers Age and Profession distribution')
visual_5.update_layout(yaxis={'categoryorder':'total ascending'})
visual_5.show()
del visual_grp_5 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_6"> Visual 6 : Distribution of Machine Learning Experience</a>

In [ ]:
visual_grp_6 = dg_c_2021.groupby(['Experience'])['Numbers'].sum().reset_index()
visual_6 = px.bar(data_frame=visual_grp_6,y='Experience',x='Numbers'
                  ,title='Kagglers Experience distribution',text='Numbers')
visual_6.update_traces(marker_color='rgb(150,202,25)') #This brings in the color for the bars
visual_6.update_layout(yaxis={'categoryorder':'total descending'})
visual_6.show()
del visual_grp_6 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_7">Visual 7: Salary Range</a>

In [ ]:
visual7 = responses_df_2021.loc[:,['Q25','Time from Start to Finish (seconds)']]
visual_grp_7 = visual7.groupby('Q25')['Time from Start to Finish (seconds)'].count().reset_index()
visual_7 = px.bar(data_frame=visual_grp_7,y='Q25',x='Time from Start to Finish (seconds)'
                  ,title='Kagglers Salary distribution')
visual_7.update_layout(yaxis={'categoryorder':'total ascending'})
visual_7.show()
del visual_grp_7,visual7 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_8"> Visual_8 Salary Distribution based on the profession</a>

In [ ]:
visual_grp_8 = dg_c_2021.groupby(['Employment','salary'])['Numbers'].count().reset_index()
visual_8 = px.bar(data_frame=visual_grp_8,y='salary',x='Numbers',color='Employment',
                  title='Salary based on profession distribution')
visual_8.update_layout(yaxis={'categoryorder':'total ascending'},height=800)
visual_8.show()
del visual_grp_8 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_9">Visual 9: Kagglers and their Industry types</a> 

In [ ]:
visual_grp_9 = dg_c_2021.groupby(['Industry','Total_Employee'])['Numbers'].count().reset_index()
visual_9 = px.bar(data_frame=visual_grp_9,x='Industry',y='Numbers',color='Total_Employee',
                  title='Industry & Total Employee distribution')
visual_9.update_layout(legend=dict(x=0,y=-0.5,traceorder="reversed",orientation="h"),#This makes Legend Horizontal
                       xaxis={'categoryorder':'total ascending'},width=1000,height=800)
visual_9.show()
del visual_grp_9 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_10">Visual 10: Primary Databased tools use Distribution</a>

In [ ]:
visual_grp_10 = Q32_A_df.groupby(['Reply_Choices'])['Counts'].sum().reset_index()
visual_10 = px.bar(data_frame=visual_grp_10,x='Counts',y='Reply_Choices',
                   title='Database usage distribution')
visual_10.update_layout(yaxis={'categoryorder':'total ascending'},height=700)
visual_10.show()
del visual_grp_10 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_11">Visual 11 : Distribution of Business Intelligence tools</a>

In [ ]:
visual_grp_11 = Q34_df.groupby(['Reply_Choices'])['Counts'].sum().reset_index()
visual_11 = px.bar(data_frame=visual_grp_11,x='Counts',y='Reply_Choices',
                   title='Business Intelligence usage distribution')
visual_11.update_layout(yaxis={'categoryorder':'total ascending'},height=700)
visual_11.show()
del visual_grp_11 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# <a id="vis_12">Visual 12 : Population Distribution with the rendering of Table in same chart</a>

In [ ]:
visual_grp_12 = dg_c_2021.groupby(['Country','Gender'])['Numbers'].sum().reset_index()
table_12 = pd.pivot_table(data=visual_grp_12,index='Country',columns='Gender',values='Numbers').fillna('-').reset_index()
table_12.sort_values(by='Man',ascending=False,inplace=True)
table_12.head()

In [ ]:
header = visual_grp_12.Gender.unique()
header = np.insert( header, 0, 'country') #This took considerable trial and error to understand
visual_12 = make_subplots(rows=2, cols=1,
                          vertical_spacing=0.1,
                          specs=[[{"type": "Bar"}],[{"type": "table"}]])

visual_12.add_trace(go.Table(header=dict(values=header,
                                         font=dict(size=10),
                                         align="left"),
                             cells=dict(values=[table_12[k].tolist() for k in table_12.columns],
                                        align = "left")),row=2, col=1)
for k in table_12.columns[1:]:
    visual_12.add_trace(go.Bar(x=table_12.Country,y=table_12[k],name=k),row=1, col=1)                                                

visual_12.update_layout(height=800,showlegend=False,
                        title_text="Countrywise Distribution",barmode='stack')
visual_12.show() 
del visual_grp_12

 [Back to Contents](#cont)

# <a id="vis_13">Visual 13: Global map of Kaggle participants</a>

In [ ]:
visual_13_grp = dg_c_2021.groupby('Country')['Numbers'].sum().reset_index()
visual_13 = px.scatter_geo(visual_13_grp, locations="Country",color='Country',
                         locationmode='country names',size='Numbers')
visual_13.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1),
                        margin={"r":0,"t":0,"l":0,"b":0},width = 1000)
visual_13.show()
del visual_13_grp

 [Back to Contents](#cont)

# <a id="vis_14">Visual 14 : Treemap of the Kaggler distribution based on the Gender</a>

In [ ]:
visual_14_grp = dg_c_2020.groupby(['Country','Gender'])['Numbers'].sum().reset_index()
visual_14_grp.Country = visual_14_grp.Country.apply(lambda x : long_sentences_seperate(x,7))
visual_14 = px.treemap(data_frame=visual_14_grp,path=['Country','Gender'],names='Country',
                      values = 'Numbers',color='Gender',title='Participating Nations with Gender distribution')
visual_14.update_layout(height=1000,width=1000)
visual_14.show()
del visual_14_grp

 [Back to Contents](#cont)

# <a id="vis_15">Visual 15: Funnel Chart of Country citizen participation</a>

In [ ]:
visual_15_grp = dg_c_2020.groupby('Country')['Numbers'].sum().reset_index()
visual_15_grp.sort_values(by='Numbers',ascending=False,inplace=True)
visual_15_grp.Country = visual_15_grp.Country.apply(lambda x : long_sentences_seperate(x,7))


visual_15 = make_subplots(rows=1, cols=2,
                          horizontal_spacing=0.1,
                          specs=[[{"type": "table"},{"type": "funnel"}]])

visual_15.add_trace(go.Table(header=dict(values=visual_15_grp.columns,
                                         font=dict(size=10),
                                         align="left"),
                             cells=dict(values=[visual_15_grp[k].tolist() for k in visual_15_grp.columns],
                                        align = "left")),row=1, col=1)
visual_15.add_trace(go.Funnel(y=visual_15_grp.Country,x=visual_15_grp.Numbers,
                              textposition = "inside",textinfo = "value+percent initial",
                              opacity = 0.95,
                              connector = {"line": {"color": "white", "dash": "dot", "width": 3}}),row=1,col=2)
visual_15.update_layout(height=1000,width=1000)
visual_15.show()
del visual_15_grp

 [Back to Contents](#cont)

# <a id="vis_16">Visual 16 : Mix of Treemap, Ribbon chart, Funnel and Pie charts</a>

In [ ]:
visual_grp_16 = dg_c_2021.groupby(['Country','Employment'])['Numbers'].sum().reset_index()
visual_pie_16 = dg_c_2021.groupby('Employment')['Numbers'].sum().reset_index()
table_16 = pd.pivot_table(data=visual_grp_16,index='Country',columns='Employment',values='Numbers').fillna('-').reset_index()
table_16.sort_values(by='Data Scientist',ascending=False,inplace=True)
table_16.head()

In [ ]:
# The below visual took considerable amount of effort to learn. The other challenge will be, in bringing 
# treemaps inside the subplots. The treemap creation is easier in plotly express, but in Graph Objects, is 
# very labour intensive, and less intuitive
visual_16 = make_subplots(rows=2, cols=2,
                          horizontal_spacing=0.1,
                          specs=[[{"type": "scatter", "colspan": 2},None],
                                 [{"type": "pie"}, {"type": "funnel"}]])

visual_16.add_trace(go.Pie(labels=visual_pie_16.Employment,values=visual_pie_16.Numbers,hole=0.3),row=2,col=1)
visual_pie_16.sort_values(by='Numbers',ascending=False,inplace=True)

visual_16.add_trace(go.Funnel(y=visual_pie_16.Employment,x=visual_pie_16.Numbers),row=2,col=2)

for k in table_16.columns[1:]:
    visual_16.add_trace(go.Scatter(x=table_16.Country,y=table_16[k],fill='tozeroy',mode='none',name=k),row=1,col=1)

visual_16.update_layout(height=1000,width=1000)

visual_16.show()

 [Back to Contents](#cont)

# <a id="vis_17">Visual 17 : Subplots of multiple demographic spread</a>

In [ ]:
#Cleaning up the data
dg_c_2021.loc[dg_c_2021.Education.str.contains('without'),'Education'] = 'Without_Degree'
dg_c_2021.loc[dg_c_2021.Education.str.contains('Bachelor'),'Education'] = 'Bachelor_Degree'
dg_c_2021.loc[dg_c_2021.Education.str.contains('Master'),'Education'] = 'Masters_Degree'
dg_c_2021.loc[dg_c_2021.Education.str.contains('without'),'Education'] = 'Without_Degree'

#Getting the data ready for feeding into the charting function
visual_17_p1 = dg_c_2021.groupby(['Experience','Employment'])['Numbers'].sum().reset_index()
visual_17_p2 = dg_c_2021.groupby(['Experience','Education'])['Numbers'].sum().reset_index()
visual_17_p3 = dg_c_2021.groupby(['Education','Employment'])['Numbers'].sum().reset_index()
visual_17_p4 = dg_c_2021.groupby('Employment')['Numbers'].sum().reset_index()

#changing the dataframe orientation to make graphing easier
table_p1_17 = pd.pivot_table(data=visual_17_p1,index='Experience',columns='Employment',
                             values='Numbers').fillna('-').reset_index()
table_p1_17.sort_values(by='Data Scientist',ascending=False,inplace=True)

table_p2_17 = pd.pivot_table(data=visual_17_p2,index='Experience',columns='Education',
                             values='Numbers').fillna('-').reset_index()
table_p2_17.sort_values(by='Bachelor_Degree',ascending=False,inplace=True)
table_p3_17 = pd.pivot_table(data=visual_17_p3,index='Education',columns='Employment',
                             values='Numbers').fillna('-').reset_index()
table_p3_17.sort_values(by='Business Analyst',ascending=False,inplace=True)

In [ ]:
visual_17 = make_subplots(rows=2,cols=2,specs=[[{'type':'scatter'},{'type':'bar'}],
                                           [{'type':'bar'},{'type':'scatter'}]])

for cols in table_p2_17.columns[1:]:
    visual_17.add_trace(go.Bar(x=table_p2_17.Experience,
                           y=table_p2_17[cols],name=cols),row=1,col=2)
    
for exp in table_p1_17.columns[1:]:
    visual_17.add_trace(go.Scatter(x=table_p1_17.Experience,
                               y=table_p1_17[exp],
                               fill='tozeroy',mode='none',
                                name=exp),row=1,col=1)
    
for exp in table_p3_17.columns[1:]:
    visual_17.add_trace(go.Scatter(x=table_p3_17.Education,
                               y=table_p3_17[exp],
                               fill='tozeroy',mode='none',
                                name=exp),row=2,col=2)

visual_17.add_trace(go.Bar(y=visual_17_p4.Numbers,
                          x=visual_17_p4.Employment),row=2,col=1)

visual_17.update_layout(height = 1000, width = 1000,showlegend=False)
del visual_17_p4,visual_17_p3,visual_17_p2,visual_17_p1,table_p1_17,table_p2_17,table_p3_17
visual_17.show()

 [Back to Contents](#cont)

# <a id="vis_18">Visual 18: Count of Employment and Gender by Country and Experience</a>

In [ ]:
visual_grp_18 = dg_c_2021.groupby(['Country','Experience'])['Numbers'].sum().reset_index()
visual_lin_18 = dg_c_2021.groupby(['Employment','Country'])['Numbers'].sum().reset_index()
table_18_1 = pd.pivot_table(data=visual_grp_18,index='Country',columns='Experience',values='Numbers').fillna('-').reset_index()
table_18_1.sort_values(by='1-3 years',ascending=False,inplace=True)
table_18_1 = table_18_1[:12]
location = table_18_1.Country
table_18_2 = pd.pivot_table(data=visual_lin_18,index='Employment',columns='Country',values='Numbers').fillna('-').reset_index()
table_18_2 = table_18_2[location].sum()
table_18_2 = table_18_2[1:]

In [ ]:
# Adding the 2nd axis for the employment
visual_18 = make_subplots(specs=[[{"secondary_y": True}]])

for cols in table_18_1.columns[1:]:
    visual_18.add_trace(go.Bar(x=table_18_1.Country,y=table_18_1[cols],name=cols),secondary_y=False)
    
visual_18.add_trace(go.Line(x=table_18_2.index,y=table_18_2.values,
                            orientation='h',name='Employment'),secondary_y=True)
# Set x-axis title
visual_18.update_xaxes(title_text="Country")

# Set y-axes titles
visual_18.update_yaxes(title_text="<b>Experience</b> yaxis title", secondary_y=False)
visual_18.update_yaxes(title_text="<b>Employment</b> yaxis title", secondary_y=True)

visual_18.update_layout(width =1000, height = 1000)
del visual_grp_18,visual_lin_18
visual_18.show()

 [Back to Contents](#cont)

# <a id="vis_19">Visual 19: Trying the waterfall Chart (Will add Dropdown option for the countries)</a>

In [ ]:
visual_grp_19 = dg_c_2021.groupby(['Country','Experience'])['Numbers'].sum().reset_index()
visual_grp_19.sort_values(by='Numbers',ascending=False,inplace=True)
grp_19_total = visual_grp_19.groupby(['Country', 'Experience']).agg({'Numbers': ['sum']}).stb.subtotal().stb.flatten()
grp_19_total = grp_19_total.sort_values(by='Country')

# Need to create sub_data to get the countries total.
subtotal = grp_19_total[grp_19_total.Experience.str.contains('subtotal')]
subtotal.sort_values(by='Numbers_sum',ascending=True,inplace=True)
subtotal = subtotal[-15:]
sub_data = grp_19_total[grp_19_total.Country.isin(subtotal.Country)]

#After selecting the top 15 countries, then merge them
merge_19 = pd.merge(left=subtotal,right=sub_data,left_on='Country',right_on='Country',how='left')
merge_19.loc[(~merge_19.Experience_y.str.contains('subtotal')),
              'Numbers_sum_x'] = merge_19.loc[(~merge_19.Experience_y.str.contains('subtotal')),
                                               'Numbers_sum_y']
merge_19.drop(['Numbers_sum_y','Experience_x'],axis=1,inplace=True)                                          

In [ ]:
#Have to thank towardsdatascience for the below code idea. Had been trying figure this out for 2 hrs... 
#https://towardsdatascience.com/waterfall-charts-with-plotly-43822918e9eb

conditionlist = [(merge_19.Experience_y.str.contains('subtotal')),
                 (~merge_19.Experience_y.str.contains('subtotal'))]

choicelist   = ['total', 'relative']

merge_19['measure'] = np.select(conditionlist, choicelist)

In [ ]:
#Seriously UK is more stylish to pronounce, and compact too.
merge_19.loc[merge_19.Country.str.contains('Kingdom'),'Country']='UK'
merge_19.loc[merge_19.Country.str.contains('Kingdom'),'Experience_y']='UK - subtotal'

In [ ]:
visual_19  = go.Figure()
merge_19 = merge_19[merge_19.Country == 'India'].sort_values(by='Numbers_sum_x',ascending=True)
visual_19.add_trace(go.Waterfall(x = merge_19['Experience_y'], y = merge_19['Numbers_sum_x'],
                                 measure = merge_19['measure'],
                                 text = merge_19['Numbers_sum_x'],textposition = 'outside',
                                 decreasing = {"marker":{"color":"crimson","line":{"color":"lightsalmon","width":2}}},
                                 increasing = {"marker":{"color":"forestgreen","line":{"color":"lightgreen", "width":2}}},
                                 totals     = {"marker":{"color":"mediumblue"}}
               ))
visual_19.update_layout(title_text = "Waterfall of the Experience in major countries",
                        title_font = dict(size=25,family='Verdana',
                                          color='darkred'),height =1000,width = 1000)
visual_19.update_xaxes(title = 'Experience Numbers')
visual_19.update_yaxes(title = 'Countries')
visual_19.show()

 [Back to Contents](#cont)

# <a id="vis_20">Visual 20: Sankey Chart to see the flow of Kagglers in Gender</a>

In [ ]:
#Sankey charts can show interesting data/group/material flows. Here we wish to see, how the community 
#bifurcates into many categories. To begin with the how it flows into multiple gender
visual_grp_20 = dg_c_2021.groupby('Gender')['Numbers'].sum().reset_index()
visual_grp_20 = visual_grp_20.stb.subtotal().reset_index()
visual_grp_20['index'] = range(0,6)
visual_grp_20.loc[visual_grp_20.index == 5,'Gender'] = 'Total_Community'

In [ ]:
#Creating Source and Targets
source = []
source.extend(repeat(visual_grp_20.index[5],5))

target = [0,1,2,3,4]

value = visual_grp_20.Numbers.values[:-1]

label = visual_grp_20.Gender.values

link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=5)
#Building the chart Json file
data = go.Sankey(link = link,node=node)
#print(data)

visual_20 = go.Figure(data)

In [ ]:
visual_20.show()

 [Back to Contents](#cont)

In [ ]:
#Creating level two of the Sankey Chart
visual_grp_20_a = dg_c_2021.groupby(['Gender','Education'])['Numbers'].sum().reset_index()
visual_grp_20_a = visual_grp_20_a.stb.subtotal().reset_index()
#Cleaning long names
visual_grp_20_a.loc[visual_grp_20_a.Education == 'Some college/university study without earning a bachelorâ\x80\x99s degree','Education'] = 'Some_College'
visual_grp_20_a.loc[visual_grp_20_a.Education == 'No formal education past high school','Education'] = 'High_School'
visual_grp_20_a.loc[visual_grp_20_a.Education == 'I prefer not to answer','Education'] = 'Not_Answered'
visual_grp_20_a.loc[visual_grp_20_a.Gender == 'Prefer to self-describe','Gender'] = 'Self_described'

In [ ]:
#Creating labels
gen = visual_grp_20_a.Gender.unique()
edu = visual_grp_20_a.Education.unique()
#Building the Source 
label = list(edu)+list(gen)
label.append('Total')

#Creating Sources & Targets
source = [8,8,8,8,8,8,8,
          9,9,9,9,9,9,9,
          10,10,10,10,10,10,10,
          11,11,11,11,11,11,11,
          12,12,12,12,12,12,12]
target = [0,1,2,3,4,5,6,
          0,1,2,3,4,5,6,
          0,1,2,3,4,5,6,
          0,1,2,3,4,5,6,
          0,1,2,3,4,5,6]

In [ ]:
value = visual_grp_20_a.Numbers.values[:-1]
#Appending the total of the entire community

In [ ]:
#Thickness of the flow path between the source and target depends on the value that is indexed. 
link = dict(source = source, target = target, value = value)
node = dict(label = label, pad=50, thickness=5)
data = go.Sankey(link = link,node=node)
#print(data)
visual_20_a = go.Figure(data)

In [ ]:
visual_20_a.show()
del visual_grp_20, visual_grp_20_a

 [Back to Contents](#cont)

# <a id="vis_21">Visual 21: Continuing with Sankey Chart with multiple Levels</a>

In [ ]:
#Creating the tail node data.
visual_grp_21 = dg_c_2021.groupby(['Employment','salary'])['Numbers'].sum().reset_index()
#Collecting the source_node data from the dataframes and building the values list
emp_val = dg_c_2021.groupby('Employment')['Numbers'].sum().reset_index()
values = list(visual_grp_21.Numbers.values) + list(emp_val.Numbers.values) 
#Building the labels list
emp = visual_grp_21.Employment.unique()
sal = visual_grp_21.salary.unique()
#print(len(emp),len(sal))
label = list(emp)+list(sal)

In [ ]:
#Building the Source and Target Lists
source = list()
for val in range(0,13):
    source.extend(repeat(val,26))

target = list()
for val in range(13,25):
    tar_part = list(np.arange(13,26))
    target = target + tar_part

#Building the color palette and making the dictionary for the source and target nodes
color_s = ['#EBBAB5', '#FEF3C7', '#A6E3D7', '#CBB4D5','#f4a460','#bc8f8f','#cd853f',
          '#ffa500','#ba55d3','#90ee90','#f08080','#7cfc00','#ff6347']
col_link = dict(zip(visual_grp_21.Employment.unique(),color_s))

#The default node colors are sufficient, but the links can be made more interesting. How many links are there?
#There are 6 sources, and 26 outgoing links, that is 6*26 = 156. That is huge... Automate!!
#Depending on the source, the links can be colored. Register that in the dataframe itself

for employ in col_link:
    visual_grp_21.loc[visual_grp_21.Employment == employ,'color'] =  col_link[employ]

color_link = visual_grp_21.color.values

In [ ]:
#Thickness of the flow path between the source and target depends on the value that is indexed. 
link = dict(source = source, target = target, value = values,color=color_link)
node = dict(label = label, pad=50, thickness=5)

#Building the Sankey chart JSON file
data = go.Sankey(link = link,node=node)
#print(data)  Can be used to see the json file

visual_21 = go.Figure(data)

#Adding the styles, colors to the graph sheet, along with the heading.
visual_21.update_layout(hovermode = 'x',title="Designation and their Salaries",
    font=dict(size = 10, color = 'white'),height=800,
    paper_bgcolor='#fffafa')
visual_21.show()

 [Back to Contents](#cont)

# <a id="vis_22">Visual 22: Primary Cloud Computing Services used</a>

In [ ]:
visual_grp_22 = Q10_df.groupby(['Reply_Choices'])['Counts'].sum().reset_index()
visual_22 = px.bar(data_frame=visual_grp_22,x='Counts',y='Reply_Choices',
                   title='Cloud Computing Usage Distribution')
visual_22.update_layout(yaxis={'categoryorder':'total ascending'},height=700)
visual_22.show()
del visual_grp_22 #Removing the unwanted DF immediately

 [Back to Contents](#cont)

# Appendix


There are 22 visuals (as of 9th Jan) lined up using the Kaggle 2021 survey data. Visuals were created to show, how easy it is to make, BI visualisation very easily with Python and Plotly. How well did it work?

It is phenomenal to see the visualisation come to life with a bit of programming knowledge. Most of the single plot visuals were walk in the park. The challenge arises when the subplots needed to be rendered. 

Treemap is super easy when using Plotly Express, but to render it as a subplot the Graph Objects have to be used. That is a biggest challenge that has taken most of time. Me being a beginner in Programming made it further challenging. (This is a work in progress)

The data is also useful for providing many quora answers, with data. Many out there, are still at the beginning stages of learning the power of Data. Need to thank Kaggle for sharing this data openly, so we can contribute by mining it.